In [321]:
import spacy
import tqdm
import pandas as pd
from pyserini.search import SimpleSearcher
from collections import defaultdict

In [322]:
nlp = spacy.load('en_core_web_sm')
searcher = SimpleSearcher('indexes/lucene-index-msmarco-doc')

In [453]:
test_set = 'eval'
method = 'glove'
queries_to_new_score = defaultdict(dict)

In [463]:
result_file = f'intermediate_results/{test_set}_desm_scores_{method}.tsv'
intermediate_res = pd.read_csv(result_file, sep='\t', header=None)
for _, (q_id, doc_id, score) in intermediate_res.iterrows():
    queries_to_new_score[str(q_id)][doc_id] = score

In [466]:
### FOR BERT ###

weights = [1, 1, 1]
intermediate_result_file = f'intermediate_results/full_results_{test_set}_set_distilbert.tsv'
with open(intermediate_result_file) as f:
    queries_to_new_score = defaultdict(dict)
    lines = f.readlines()
    for line in lines:
        tokens = line.split()
        q_id, doc_id = tokens[:2]
        scores = list(map(float, tokens[2:]))
        final_score = 0
        total_weight = 0
        for i, top_score in enumerate(scores):
            final_score += top_score * weights[i]
            total_weight += weights[i]
        final_score = final_score / total_weight if total_weight != 0 else 0
        queries_to_new_score[q_id][doc_id] = final_score

In [461]:
queries_to_docs = defaultdict(list)
queries_to_bm25_score = defaultdict(dict)
docs_to_text = {}
with open('results/dev-bm25-100.trec') as f, open(f'rerank_results/{test_set}initial_scores.tsv', 'w') as g:
    lines = f.readlines()
    for line in tqdm.tqdm(lines):
        query_id, _, doc_id, rank, score, _ = line.split()
        if query_id in queries_to_new_score:
            queries_to_docs[query_id].append(doc_id)
            queries_to_bm25_score[query_id][doc_id] = float(score)
            docs_to_text[doc_id] = searcher.doc(doc_id).contents().replace('\n', ' ')
            g.write(f'{query_id} Q0 {doc_id} {rank} {score} {"bm25"}\n')

100%|██████████| 519300/519300 [00:06<00:00, 81867.75it/s] 


In [462]:
print(queries_to_bm25_score['1104031']['D2401591'])
print(queries_to_new_score['1104031']['D2401591'])

KeyError: 'D2401591'

In [467]:
alpha = 0.5
for query_id, docs in queries_to_docs.items():
    queries_to_docs[query_id] = sorted(docs, key=lambda d: -(1 - alpha) * queries_to_bm25_score[query_id][d] - alpha * queries_to_new_score[query_id][d])

In [468]:
with open(f'rerank_results/final_scores_{test_set}_distilbert.tsv', 'w') as f:
    for query_id, docs in queries_to_docs.items():
        for idx, doc_id in enumerate(docs):
            f.write(f'{query_id} Q0 {doc_id} {idx} {1 / (idx + 1)} bert\n')